# TensorFlow Framework - Keras API

## Classification hello world program

### Steps for classification  
* Data
    - Separate Training
    - Separate Validation
    - Separate Testing data
  
* Model
    - Define Inputs
    - Define Layers
    - Define Outputs
  
* Train
    - Choose loss function
    - Choose optimizer
    - Choose metrics
  
* Evaluate and visualize
    - Evaluate using the testing data
    - Visualze reuslts

##### Tutorial by Matt Adiletta

In [1]:
! python3 --version

Python 3.6.3


In [24]:
import site
site.addsitedir("/usr/local/lib/python3.6.3/site-packages/")

In [25]:
import os
import numpy as np

from pprint import pprint

In [26]:
os.system("clear")

0

In [27]:
!pip3 install tensorflow

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### Data

In [28]:
from tensorflow.keras.datasets import mnist

In [91]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path="mnist.npz")

In [92]:
# SSL Certificate verify failed issue solved
# Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1051)
# https://stackoverflow.com/questions/42098126/mac-osx-python-ssl-sslerror-ssl-certificate-verify-failed-certificate-verify
! open /Applications/Python\ 3.6/Install\ Certificates.command

# https://github.com/tensorflow/tensorflow/issues/33285
import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [93]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path="mnist.npz")

In [94]:
# 60,000 trainning images, size: 28x28
print("x train shape: ", np.shape(x_train))
print("y train shape: ", np.shape(y_train))

x train shape:  (60000, 28, 28)
y train shape:  (60000,)


In [95]:
# Separate validation
percent_validation = 10
val_index = int(np.shape(x_train)[0] * percent_validation / 100)
print(val_index)

6000


In [96]:
x_val, x_train = x_train[0:val_index], x_train[val_index:]
y_val, y_train = y_train[0:val_index], y_train[val_index:]
print("x train shape: ", np.shape(x_train))
print("y train shape: ", np.shape(y_train))

x train shape:  (54000, 28, 28)
y train shape:  (54000,)


In [97]:
# Visualize an image
from PIL import Image
img = Image.fromarray(x_train[0], 'L').resize((512, 512))
img.show()

In [98]:
print("y value: ",y_train[0])

y value:  6


In [99]:
# One hot encoding for Y label
def one_hot(y_in, num_encodings):
    y_out = np.zeros( (np.shape(y_in)[0], num_encodings) )
    for idx, element in enumerate(y_in):
        y_out[idx][element] = 1
    
    return y_out

y_tr = one_hot(y_train, 10)
y_val = one_hot(y_val, 10)
y_te = one_hot(y_test, 10)

print("y_tr one-hot: ", y_tr[0])

y_tr one-hot:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [100]:
print("x train shape: ", np.shape(x_train))

x train shape:  (54000, 28, 28)


In [101]:
# Need to make the train matrix homogeneous.
X_tr = np.expand_dims(x_train, axis=3)
X_val = np.expand_dims(x_val, axis=3)
X_te = np.expand_dims(x_test, axis=3)

#### Model

In [102]:
def create_model(width, height):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
    
    inputs = Input(shape=(width, height, 1)) # Homogeneous
    
    # VGG: 4x(conv -> maxpool -> dropout) -> flatten -> dense -> output
    # https://keras.io/api/layers/convolution_layers/convolution2d/
    # https://keras.io/api/layers/activations/
    # Conv2D works as a lambda func, inputs is an arg
    conv1 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(inputs)
    maxpool1 = MaxPooling2D(pool_size=2)(conv1)
    dropout1= Dropout(0.3)(maxpool1)
    
    conv2 = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(dropout1)
    maxpool2 = MaxPooling2D(pool_size=2)(conv2)
    dropout2 = Dropout(0.3)(maxpool2)
    
    conv3 = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")(dropout2)
    maxpool3 = MaxPooling2D(pool_size=2)(conv3)
    dropout3 = Dropout(0.3)(maxpool3)
    
    conv4 = Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu")(dropout3)
    maxpool4 = MaxPooling2D(pool_size=2)(conv4)
    
    flatten = Flatten()(maxpool4)
    dense1 = Dense(256, activation="relu")(flatten)
    output = Dense(10, activation="softmax")(dense1)

    return Model(inputs=inputs, outputs=output)

model = create_model(28, 28)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 7, 7, 32)          0         
__________

#### Train

In [103]:
from my_callback import CustomCallback

model.compile(
    Loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.fit(
    X_tr,
    y_tr,
    visualization_data=(X_val, y_val),
    batch_size=256,
    epochs=5,
    shuffle=True,
    verbose=0,
    callbacks=[CustomCallback(model, X_tr, y_tr, X_val, y_val)]
)

ModuleNotFoundError: No module named 'my_callback'

#### Evaluate